In [21]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # @param ["tensorflow", "jax", "torch"]

import keras
from keras import layers
from keras import ops
from keras.preprocessing import image_dataset_from_directory

import numpy as np
import matplotlib.pyplot as plt

In [22]:
# Path to your dataset in Google Drive
train_dir = '/content/drive/MyDrive/APS360 Project/Data/train'
test_dir = '/content/drive/MyDrive/APS360 Project/Data/test'

# Load the training and testing datasets
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(32, 32),  # Resizing to the original CIFAR-100 size
    batch_size=batch_size,
    label_mode='int',  # Assuming integer labels
    shuffle=True
)

test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(32, 32),  # Resizing to the original CIFAR-100 size
    batch_size=batch_size,
    label_mode='int',  # Assuming integer labels
    shuffle=False
)

# Optionally, you can also create a validation split
#val_dataset = train_dataset.take(5000 // batch_size)  # Assuming 10% validation
#train_dataset = train_dataset.skip(5000 // batch_size)

Found 28710 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.


In [23]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 10  # For real training, use num_epochs=100. 10 is a test value
image_size = 72  # We'll resize input images to this size
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [
    2048,
    1024,
]  # Size of the dense layers of the final classifier

In [24]:
data_augmentation = keras.Sequential(
    [
        layers.Resizing(image_size, image_size),  # Resize to the new image size
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="data_augmentation",
)

In [25]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=keras.activations.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [26]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        input_shape = ops.shape(images)
        batch_size = input_shape[0]
        height = input_shape[1]
        width = input_shape[2]
        channels = input_shape[3]
        num_patches_h = height // self.patch_size
        num_patches_w = width // self.patch_size
        patches = keras.ops.image.extract_patches(images, size=self.patch_size)
        patches = ops.reshape(
            patches,
            (
                batch_size,
                num_patches_h * num_patches_w,
                self.patch_size * self.patch_size * channels,
            ),
        )
        return patches

    def get_config(self):
        config = super().get_config()
        config.update({"patch_size": self.patch_size})
        return config

In [27]:
plt.figure(figsize=(4, 4))
image = train_dataset[np.random.choice(range(train_dataset.shape[0]))]
plt.imshow(image.astype("uint8"))
plt.axis("off")

resized_image = ops.image.resize(
    ops.convert_to_tensor([image]), size=(image_size, image_size)
)
patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = ops.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(ops.convert_to_numpy(patch_img).astype("uint8"))
    plt.axis("off")

AttributeError: '_PrefetchDataset' object has no attribute 'shape'

<Figure size 400x400 with 0 Axes>

In [28]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = ops.expand_dims(
            ops.arange(start=0, stop=self.num_patches, step=1), axis=0
        )
        projected_patches = self.projection(patch)
        encoded = projected_patches + self.position_embedding(positions)
        return encoded

    def get_config(self):
        config = super().get_config()
        config.update({"num_patches": self.num_patches})
        return config

In [ ]:
def run_experiment(model):
    optimizer = keras.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint.weights.h5"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history


vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)


def plot_history(item):
    plt.plot(history.history[item], label=item)
    plt.plot(history.history["val_" + item], label="val_" + item)
    plt.xlabel("Epochs")
    plt.ylabel(item)
    plt.title("Train and Validation {} Over Epochs".format(item), fontsize=14)
    plt.legend()
    plt.grid()
    plt.show()


plot_history("loss")
plot_history("top-5-accuracy")

Epoch 1/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 3217s 18s/step - accuracy: 0.0185 - loss: 5.3402 - top-5-accuracy: 0.0787 - val_accuracy: 0.0406 - val_loss: 4.3767 - val_top-5-accuracy: 0.1642
Epoch 2/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 3170s 18s/step - accuracy: 0.0336 - loss: 4.3873 - top-5-accuracy: 0.1377 - val_accuracy: 0.0642 - val_loss: 4.1696 - val_top-5-accuracy: 0.2258
Epoch 3/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 3117s 18s/step - accuracy: 0.0458 - loss: 4.2378 - top-5-accuracy: 0.1868 - val_accuracy: 0.0766 - val_loss: 4.0795 - val_top-5-accuracy: 0.2554
Epoch 4/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 3093s 17s/step - accuracy: 0.0582 - loss: 4.1420 - top-5-accuracy: 0.2156 - val_accuracy: 0.0870 - val_loss: 3.9892 - val_top-5-accuracy: 0.2884
Epoch 5/10
 44/176 ━━━━━━━━━━━━━━━━━━━━ 36:53 17s/step - accuracy: 0.0748 - loss: 4.0326 - top-5-accuracy: 0.2590

In [29]:
def create_vit_classifier():
    inputs = keras.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model